In [ ]:
import matplotlib.pyplot as plt

# Spotify colors
BG = "#e1ece3"
PRIMARY = "#62d089"
EMPHASIS = "#457e59"
GRID = "#a8b2a8"

plt.rcParams.update({
    "figure.facecolor": BG,
    "axes.facecolor": BG,
    "axes.edgecolor": BG,
    "axes.labelcolor": "#2b2b2b",
    "xtick.color": "#2b2b2b",
    "ytick.color": "#2b2b2b",
    "grid.color": GRID,
    "grid.alpha": 0.4,
    "axes.grid": True,
    "font.size": 11
})

df = pd.read_csv('../data/spotify_dedup.csv')

# Create comparison plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.patch.set_facecolor(BG)

model_names = list(models.keys())
train_r2 = [results_df[results_df['Model'] == name]['Train R²'].values[0] for name in model_names]
test_r2 = [results_df[results_df['Model'] == name]['Test R²'].values[0] for name in model_names]
rmse = [results_df[results_df['Model'] == name]['RMSE'].values[0] for name in model_names]

colors = ['#a8d5ba', PRIMARY, EMPHASIS]

# Plot 1: R² Comparison
x = np.arange(len(model_names))
width = 0.35

axes[0].bar(x - width/2, train_r2, width, label='Train R²', color=PRIMARY, alpha=0.8, edgecolor='white', linewidth=1.5)
axes[0].bar(x + width/2, test_r2, width, label='Test R²', color=EMPHASIS, alpha=0.8, edgecolor='white', linewidth=1.5)
axes[0].set_xlabel('Models', fontweight='bold')
axes[0].set_ylabel('R² Score', fontweight='bold')
axes[0].set_title('R² Score Comparison', fontweight='bold', pad=15)
axes[0].set_xticks(x)
axes[0].set_xticklabels(model_names, rotation=15, ha='right')
axes[0].legend(framealpha=0.95).get_frame().set_facecolor(BG)

# Plot 2: RMSE Comparison
axes[1].bar(model_names, rmse, color=colors, alpha=0.8, edgecolor='white', linewidth=1.5)
axes[1].set_xlabel('Models', fontweight='bold')
axes[1].set_ylabel('RMSE', fontweight='bold')
axes[1].set_title('RMSE Comparison (Lower is Better)', fontweight='bold', pad=15)
axes[1].set_xticklabels(model_names, rotation=15, ha='right')

# Plot 3: Overfitting Analysis
overfit = [t - te for t, te in zip(train_r2, test_r2)]
axes[2].bar(model_names, overfit, color=colors, alpha=0.8, edgecolor='white', linewidth=1.5)
axes[2].axhline(y=0.05, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Overfit Threshold')
axes[2].set_xlabel('Models', fontweight='bold')
axes[2].set_ylabel('Train R² - Test R²', fontweight='bold')
axes[2].set_title('Overfitting Analysis (Lower is Better)', fontweight='bold', pad=15)
axes[2].set_xticklabels(model_names, rotation=15, ha='right')
axes[2].legend(framealpha=0.95).get_frame().set_facecolor(BG)

plt.tight_layout()
plt.show()

## ✅ Summary & Key Takeaways

### Model Performance Linear Regression VS Random Forest

**Best Configuration**:
- Feature Set: **Top 12 features**
- R² Score: **0.457** (45.7% variance explained)
- RMSE: **15.07** popularity points
- MAE: **10.46** popularity points

**Hyperparameters**:
```python
{
    'n_estimators': 200,
    'max_depth': None,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'sqrt'
}
```

### Comparison with Linear Regression (Notebook 08)

| Metric | Linear (Ridge) | Random Forest | Improvement |
|--------|---------------|---------------|-------------|
| R²     | 0.3231        | 0.4571        | **+41.5%** |
| RMSE   | 16.83         | 15.07         | **-10.5%** |
| MAE    | ~11.57*       | 10.46         | **-9.6%** |

*Estimated from your linear regression output

### Key Findings

**1. Non-Linear Relationships Exist**
- Random Forest captures **non-linear patterns** missed by linear models
- 40%+ improvement in R² confirms complex interactions between features
- Example: Genre × Audio features may have non-additive effects

**2. Feature Importance Differs from Linear Coefficients**
- **Linear Model** top features (from your output):
  1. track_genre (11.57)
  2. speechiness (-10.09)
  3. danceability (7.28)

- **Random Forest** top features:
  1. track_genre (77%!)
  2. acousticness (2.4%)
  3. danceability (2.0%)

- RF shows **genre dominance** more clearly (77% vs 11.57 coefficient)
- Acousticness important in RF but not in linear model (non-linear effect?)

**3. Feature Selection Had Minimal Impact**
- Full: R² = 0.452
- Reduced: R² = 0.456
- Top: R² = 0.457
- Only ~1% difference between all three sets
- **Practical implication**: Use Top features (50% fewer with same performance)

**4. Model Limitations**
- Still explains only ~46% of variance
- Remaining 54% likely due to:
  - External factors (marketing, radio play, playlist placement)
  - Temporal trends (release timing, viral moments)
  - Subjective taste (cultural context, demographics)
  - Artist popularity (brand recognition)

### Next Steps (Notebook 10: Stacking Regressor)

**Ensemble Strategy**:
1. Combine Linear Regression + Random Forest predictions
2. Use meta-learner to weight each model optimally
3. Expected benefits:
   - Linear model: Good for simple relationships
   - Random Forest: Good for complex patterns
   - Stacking: Best of both worlds

**Expected Results**:
- R² improvement: ~2-5% (historically typical for stacking)
- Target: R² ≈ 0.47-0.50 if successful

### Recommendations for Production

**1. Use Top Feature Set**
- 12 features instead of 24 (50% reduction)
- Faster predictions (important for real-time applications)
- Same performance as full set

**2. Consider Genre-Specific Models**
- Genre explains 77% of RF importance
- Train separate models per genre for better predictions
- Example: Pop model vs Metal model (different popularity patterns)

**3. Feature Engineering Opportunities**
- Current features plateau at ~46% R²
- Consider adding:
  - Artist popularity score
  - Release date features (day of week, month, year)
  - Playlist inclusion count
  - Social media metrics

**4. Hyperparameter Insights**
- `max_depth=None` worked best (no overfitting)
- Suggests dataset is large enough to support complex trees
- `n_estimators=200` marginally better than 100
- Diminishing returns beyond 200 trees (not tested but typical)

### Time Complexity Note
- Training time: ~33 minutes total (11 min × 3 feature sets)
- 120 fits per set (24 configs × 5 folds)
- Prediction time: <1 second for 30,000+ songs
- Acceptable for most production scenarios